<a href="https://colab.research.google.com/github/carlos-alves-one/-Crypto-Electronic-Medical-Records/blob/main/crypto_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Blockchain Programming
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### Programming Assignment

## Import Cryptography Libraries

In [19]:
# Import the 'hashes' module from the cryptography library's hazmat primitives, used for cryptographic hashing
from cryptography.hazmat.primitives import hashes

# Import 'Encoding', 'PublicFormat', and 'load_der_public_key' from the serialization module for encoding formats,
# public key formats, and loading DER-encoded public keys, respectively
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat, load_der_public_key

# Import the 'ec' (Elliptic Curve cryptography) and 'utils' modules from the asymmetric part of the hazmat primitives
# for performing operations with elliptic curves and various utility functions
from cryptography.hazmat.primitives.asymmetric import ec, utils


# Part 1: Records and Verification

The objective of the first phase of this project is to establish a framework for Electronic Medical Records (EMR) that leverages cryptographic techniques, marking a significant leap forward in creating secure digital infrastructures, especially within the healthcare industry. This phase delves into four key areas: the construction of classes, the application of cryptography, the management of errors and validations, and the formulation of testing protocols. This effort’s foundation is the meticulous design of classes to ensure that the EMR system is equipped with all the essential features and functions for effective operation. Employing cryptographic algorithms, primarily via the pyca/cryptography library, is pivotal for protecting medical records from unauthorized access and modifications. Implementing robust error handling and validation processes is indispensable for detecting and addressing discrepancies such as invalid signatures or tampered records. Furthermore, developing detailed testing strategies is crucial for confirming the system’s functionality and security safeguards, mitigating potential risks, and upholding the system’s integrity. By focusing on these critical aspects, the development of EMR systems significantly enhances the protection of patient data, driving the healthcare sector toward a more secure and streamlined digital era.

## EMR Class

The EMR class embodies a comprehensive system for Electronic Medical Records, integrating essential functionalities for initialising records and assuring their integrity and authenticity. It is designed to perform critical operations such as hash validation, prescription validation, nonce verification, transaction ID (txid) calculation, and digital signature verification. This class encapsulates the logic for creating, verifying, and manipulating electronic medical records. It includes methods for initialising a record, verifying its integrity and authenticity, and computing necessary cryptographic hashes and signatures, ensuring secure and reliable medical data management.

In [20]:
class EMR:

    # Declare the constructor for this class
    def __init__(self, Dr_hash, Patient_hash, Dr_public_key, prescription, nonce, signature, txid):

        # Initialize the EMR instance with doctor's and patient's hashes, doctor's public key,
        # prescription details, nonce, digital signature, and transaction ID
        self.Dr_hash = Dr_hash
        self.Patient_hash = Patient_hash
        self.Dr_public_key = Dr_public_key
        self.prescription = prescription
        self.nonce = nonce
        self.signature = signature
        self.txid = txid

    # Declare function to perform several checks to verify the integrity and authenticity of the EMR data
    def verify(self, Dr_previous_nonce):

        # Check the length of doctor's and patient's hashes
        if len(self.Dr_hash) != 20 or len(self.Patient_hash) != 20:
            raise Exception("Hash is wrong length")

        # Validate the doctor's public key by comparing its hash against the provided doctor's hash
        if calculate_sha1_hash(self.Dr_public_key) != self.Dr_hash:
            raise Exception("Invalid doctor public key")

        # Ensure the prescription text is valid and within the allowed byte size
        if not isinstance(self.prescription, str) or len(self.prescription.encode('utf-8')) > 200:
            raise Exception("Invalid prescription")

        # Check if the nonce is sequentially correct
        if self.nonce != Dr_previous_nonce + 1:
            raise Exception("Invalid nonce")

        # Validate the transaction ID by recalculating it and comparing with the provided value
        expected_txid = calculate_txid(self.Dr_hash, self.Patient_hash, self.Dr_public_key, self.prescription, self.nonce, self.signature)
        if self.txid != expected_txid:
            raise Exception("Invalid txid")

        # Verify the digital signature to ensure the data's integrity and authenticity
        signature_hash = calculate_signature_hash(self.Patient_hash, self.prescription, self.nonce)
        key = load_der_public_key(self.Dr_public_key)
        try:
            key.verify(self.signature, signature_hash, ec.ECDSA(utils.Prehashed(hashes.SHA256())))
        except:
            raise Exception("Invalid signature")


### Supporting Functions

> This function calculate_sha1_hash computes the SHA-1 hash of the provided data. SHA-1 generates a 160-bit (20-byte) hash, which is part of various security protocols and systems.

In [21]:
def calculate_sha1_hash(data):
    digest = hashes.Hash(hashes.SHA1())
    digest.update(data)
    return digest.finalize()


> This function calculate_txid generates a transaction ID using SHA-256 by hashing together the doctor's hash, patient's hash, doctor's public key, prescription details, nonce, and digital signature.

In [22]:
def calculate_txid(Dr_hash, Patient_hash, Dr_public_key, prescription, nonce, signature):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(Dr_hash)
    digest.update(Patient_hash)
    digest.update(Dr_public_key)
    digest.update(prescription.encode('utf-8'))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    digest.update(signature)
    return digest.finalize()


> This function calculate_signature_hash produces a hash used for signature verification, combining the patient's hash, prescription, and nonce using SHA-256.

In [23]:
def calculate_signature_hash(Patient_hash, prescription, nonce):
    digest = hashes.Hash(hashes.SHA256())
    digest.update(Patient_hash)
    digest.update(prescription.encode('utf-8'))
    digest.update(nonce.to_bytes(8, byteorder='little', signed=False))
    return digest.finalize()


These components together form a secure system for managing and verifying electronic medical records, ensuring that they are authentic, have not been tampered with, and come from a verified source. The verification process involves checking the integrity of the provided information, such as hash lengths, the correctness of the nonce, the validity of the transaction ID, and the authenticity of the signature using an elliptic curve digital signature algorithm (ECDSA) with SHA-256 as the hash function.

## Create Signed Record Function

This function is designed to generate a signed EMR (Electronic Medical Record) record, leveraging the doctor's private key alongside the patient's public key hash, the prescription, and a nonce as inputs. It meticulously prepares the necessary data, computes the signature, and assembles the record. The process includes calculating the transaction ID (txid) and the signature for the new record, ensuring the integrity and authenticity of the medical record through these cryptographic measures.

In [24]:
def create_signed_record(Dr_private_key, patient_hash, prescription, nonce):

    # Convert the doctor's public key to DER format
    Dr_public_key = Dr_private_key.public_key().public_bytes(encoding=Encoding.DER, format=PublicFormat.SubjectPublicKeyInfo)

    # Calculate the doctor's public key hash
    Dr_hash = calculate_sha1_hash(Dr_public_key)

    # Prepare the data for signing
    signature_hash = calculate_signature_hash(patient_hash, prescription, nonce)

    # Sign the data
    signature = Dr_private_key.sign(signature_hash, ec.ECDSA(utils.Prehashed(hashes.SHA256())))

    # Calculate the transaction ID
    txid = calculate_txid(Dr_hash, patient_hash, Dr_public_key, prescription, nonce, signature)

    # Create and return the EMR record
    return EMR(Dr_hash, patient_hash, Dr_public_key, prescription, nonce, signature, txid)


## Test Cases

In [25]:
# Import the unittest module for creating and running unit tests in Python
import unittest


This code snippet defines a test suite for the EMR class that verifies the correct behaviour of the class and the function create_signed_record, focusing on the verification logic under various conditions, using Python's unit test framework adapted for execution in an environment that requires a different approach than running tests in a standard Python environment due to the interactive nature of notebooks.

In [26]:
class TestEMR(unittest.TestCase):

    # Define a setup method to initialize variables before each test case runs
    def setUp(self):

        # Generate a doctor's private key using the SECP256K1 elliptic curve
        self.Dr_private_key = ec.generate_private_key(ec.SECP256K1())

        # Calculate and store a SHA-1 hash of a simulated patient's public key
        self.patient_hash = calculate_sha1_hash(b"Patient's Public Key")

        # Define a sample prescription text for testing
        self.prescription = "Medication A: 10mg"

        # Initialize a nonce to 1, to be used in the creation of a test EMR record
        self.nonce = 1

    # Define a test case method for verifying a successfully signed record
    def test_record_verification_success(self):

        # Create a signed EMR record using the doctor's private key, patient hash, prescription data, and nonce
        record = create_signed_record(self.Dr_private_key, self.patient_hash, self.prescription, self.nonce)

        # Set the previous nonce value to 0, assuming this is the first transaction record
        Dr_previous_nonce = 0

        # Attempt to verify the record; set result to True if successful, False if an exception occurs
        try:
            record.verify(Dr_previous_nonce)
            result = True
        except Exception as e:
            result = False

        # Assert that the result is True, indicating the record verification was successful
        self.assertTrue(result)

    # Define a test case to verify that an altered transaction ID causes record verification to fail
    def test_invalid_txid_raises_exception(self):

        # Create a signed EMR record with valid parameters
        record = create_signed_record(self.Dr_private_key, self.patient_hash, self.prescription, self.nonce)

        # Manually set the record's transaction ID (txid) to an invalid value
        record.txid = b"Invalid TXID"

        # Use a context manager to assert that an exception is raised during record verification, indicating failure
        with self.assertRaises(Exception):

            record.verify(0)  # Attempt to verify the record with a modified txid, expecting it to raise an exception

    # Define a test case to ensure that tampering with the signature causes record verification to fail
    def test_invalid_signature_raises_exception(self):

        # Create a new EMR record with a valid signature initially
        record = create_signed_record(self.Dr_private_key, self.patient_hash, self.prescription, self.nonce)

        # Manually change the record's signature to an invalid value to simulate tampering
        record.signature = b"Invalid Signature"

        # Use a context manager to assert that an exception is raised, indicating that the record's verification fails
        with self.assertRaises(Exception):

            record.verify(0)  # Attempt to verify the tampered record, expecting failure due to the invalid signature


In [27]:
# Load all test cases from the TestEMR class into a test suite
suite = unittest.TestLoader().loadTestsFromTestCase(TestEMR)


In [28]:
# Execute the test suite with a higher verbosity level for detailed output
unittest.TextTestRunner(verbosity=2).run(suite)


test_invalid_signature_raises_exception (__main__.TestEMR) ... ok
test_invalid_txid_raises_exception (__main__.TestEMR) ... ok
test_record_verification_success (__main__.TestEMR) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.025s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>